In [1]:
# Dependencies
import requests
import json
import pandas as pd
import time
from geopy.geocoders import GoogleV3

# Google developer API key
from config import gkey

sorted_agencies_grid = 'agencies_df_grid_drop.csv'
sorted_grid_df = pd.read_csv(sorted_agencies_grid)
sorted_grid_df



,Unnamed: 0,agency_name,agency_status,agency_address,agency_latitude,agency_longitude
0,0,Acceptance Insurance,OPERATIONAL,"3000 Dunn Avenue Suite 68A, Jacksonville",30.437619,-81.700565
1,1,Phenix Supply Co,OPERATIONAL,"11705 Industry Drive, Jacksonville",30.449305,-81.625894
2,2,Insurance Auto Auctions,OPERATIONAL,"14492 New Kings Road, Jacksonville",30.496374,-81.789270
3,3,Florida State College of Jacksonville: Insuran...,OPERATIONAL,"3939 Roosevelt Boulevard, Jacksonville",30.300482,-81.719044
4,4,Gary Anderson - State Farm Insurance Agent,OPERATIONAL,"450111 State Road 200, Callahan",30.568916,-81.825901
...,...,...,...,...,...,...
1397,1397,"A.W. Adams Insurance, LLC",OPERATIONAL,"30 Commerce Street, Clayton",34.877613,-83.398781
1398,1398,Allstate,OPERATIONAL,"30 Commerce Street, Clayton",34.877611,-83.398783
1399,1399,Bynum Insurance Agency,OPERATIONAL,"151 Hiawassee Street, Clayton",34.879789,-83.403506
1400,1400,Norton Mountain Insurance,OPERATIONAL,"855 Washington Street, Clarkesville",34.608675,-83.522450


In [2]:
# set the pause duration between api requests
pause = 0.1

In [5]:
# create a column to put lat-long into the format google likes - this just makes it easier to call their API
sorted_grid_df['latlng'] = sorted_grid_df.apply(lambda row: '{},{}'.format(row['agency_latitude'], row['agency_longitude']), axis=1)
sorted_grid_df

,Unnamed: 0,agency_name,agency_status,agency_address,agency_latitude,agency_longitude,latlng
0,0,Acceptance Insurance,OPERATIONAL,"3000 Dunn Avenue Suite 68A, Jacksonville",30.437619,-81.700565,"30.43761869999999,-81.7005653"
1,1,Phenix Supply Co,OPERATIONAL,"11705 Industry Drive, Jacksonville",30.449305,-81.625894,"30.449305,-81.62589400000002"
2,2,Insurance Auto Auctions,OPERATIONAL,"14492 New Kings Road, Jacksonville",30.496374,-81.789270,"30.49637439999999,-81.7892703"
3,3,Florida State College of Jacksonville: Insuran...,OPERATIONAL,"3939 Roosevelt Boulevard, Jacksonville",30.300482,-81.719044,"30.3004824,-81.7190443"
4,4,Gary Anderson - State Farm Insurance Agent,OPERATIONAL,"450111 State Road 200, Callahan",30.568916,-81.825901,"30.56891599999999,-81.8259006"
...,...,...,...,...,...,...,...
1397,1397,"A.W. Adams Insurance, LLC",OPERATIONAL,"30 Commerce Street, Clayton",34.877613,-83.398781,"34.8776126,-83.39878089999998"
1398,1398,Allstate,OPERATIONAL,"30 Commerce Street, Clayton",34.877611,-83.398783,"34.87761089999999,-83.398783"
1399,1399,Bynum Insurance Agency,OPERATIONAL,"151 Hiawassee Street, Clayton",34.879789,-83.403506,"34.879789,-83.40350610000002"
1400,1400,Norton Mountain Insurance,OPERATIONAL,"855 Washington Street, Clarkesville",34.608675,-83.522450,"34.6086747,-83.52245009999999"


In [ ]:
def get_city(geocode_result):
     if 'address_components' in geocode_result:
        for address_component in geocode_result['address_components']:
            if 'locality' in address_component['types']:
                return address_component['long_name']
            
def get_state(geocode_result):
     if 'address_components' in geocode_result:
        for address_component in geocode_result['address_components']:
            if 'administrative_area_level_1' in address_component['types']:
                return address_component['long_name']
            
def get_county(geocode_result):
     if 'address_components' in geocode_result:
        for address_component in geocode_result['address_components']:
            if 'administrative_area_level_2' in address_component['types']:
                return address_component['long_name']

In [ ]:
# pass the Google API latlng data to reverse geocode it
def reverse_geocode(latlng):
    time.sleep(pause)
    url = 'https://maps.googleapis.com/maps/api/geocode/json?latlng={}&key={}'
    request = url.format(latlng, gkey)
    response = requests.get(request)
    data = response.json()
    if len(data['results']) > 0:
        return data['results'][0] #if we got results, return the first result
    
geocode_results = sorted_grid_df['latlng'].map(reverse_geocode)


In [ ]:
sorted_grid_df['city'] = geocode_results.map(get_city)                
sorted_grid_df['state'] = geocode_results.map(get_state)
sorted_grid_df['county'] = geocode_results.map(get_county)
sorted_grid_df

In [ ]:
sorted_grid_df.to_csv(r'C:\Users\erinz\Desktop')